In [42]:
from google.cloud import storage

path_to_private_key = 'is3107-418903-a6116d294003.json'

storage_client = storage.Client.from_service_account_json(json_credentials_path=path_to_private_key)

In [43]:
from google.api_core.exceptions import Forbidden, NotFound

def download_blob(bucket_name,source_blob_name, destination_file_name):
    try:
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(source_blob_name)
        blob.download_to_filename(destination_file_name)
    except Forbidden as e:
        print(f"Access denied. You might have a billing or permissions issue: {e}")
    except NotFound as e:
        print(f"The requested file was not found: {e}")

    

# download models
download_blob('is3107-model', 'ml_model_test/modelGB.pkl', 'modelGB.pkl')
download_blob('is3107-model', 'ml_model_test/modelDT.pkl', 'modelDT.pkl')

# download model dataset
download_blob('is3107-model', 'ml_model_test/dataset.csv', 'dataset.csv')

In [12]:
%pip install -U scikit-learn==1.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 9.5 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.2
    Uninstalling scikit-learn-1.4.2:
      Successfully uninstalled scikit-learn-1.4.2
Note: you may need to restart the kernel to use updated packages.


In [44]:
import joblib
import pandas as pd

modelGB = joblib.load('modelGB.pkl')
modelDT = joblib.load('modelDT.pkl')
data = pd.read_csv('dataset.csv')

/Users/isabel/opt/anaconda3/envs/IS3107/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DummyRegressor from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  
/Users/isabel/opt/anaconda3/envs/IS3107/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  
/Users/isabel/opt/anaconda3/envs/IS3107/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator GradientBoostingRegressor from

In [45]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

y = data['price']
x = data.drop('price', axis=1)

grid_searchGB = GridSearchCV(modelGB, param_grid, cv=3, scoring='accuracy')
# grid_searchDT = GridSearchCV(modelDT, param_grid, cv=3, scoring='accuracy')


# Fit on your data
grid_searchGB.fit(x, y)
# grid_searchDT.fit(x, y)


# Best model
best_modelGB = grid_searchGB.best_estimator_
# best_modelDT = grid_searchDT.best_estimator_

/Users/isabel/opt/anaconda3/envs/IS3107/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/isabel/opt/anaconda3/envs/IS3107/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    cross_validate : Calculate one or more scores and timings for each CV
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/isabel/opt/anaconda3/envs/IS3107/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    score : float
           ^^^^^^^
  File "/Users/isabel/opt/anaconda3/envs/IS3107/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    ),
       
  File "/Users/isabel/opt/anaconda3/envs/IS3107/lib/python3.11/site-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    ):
  

# Predict on user input

In [56]:
data_dict = {
    'owners': 2,
    'eng_cap': 1580.0,
    'depreciation': 16954.0,
    'mileage': 111000.0,
    'power': 1036.0,
    'coe_left': 3.2,
    'omv': 27383.0,
    'arf': 5000.0,
    'model_seats': 5,
    'model_weight_kg': 1433.560091,
    'model_engine_cyl': 4,
    'model_fuel_cap_l': 45.0415,
    'years_since_reg': 7,
    'model_body_Coupe': False,
    'model_body_Hatchback': False,
    'model_body_Minivan': False,
    'model_body_SUV': True,
    'model_body_Sedan': False,
    'model_transmission_type_CVT': False,
    'model_transmission_type_Manual': True,
    'model_drive_Front Wheel Drive': True,
    'model_drive_Rear Wheel Drive': False,
    'model_engine_fuel_Premium': False,
    'model_engine_fuel_Regular': True,
    'brands': 1.0
}

In [57]:
data_df = pd.DataFrame([data_dict])

In [58]:
predictions = best_modelGB.predict(data_df)

In [60]:
predictions[0]

56801.0019509659